# Player Tracking System Comparison

This notebook compares 3 different player tracking systems:
- Eagle
- Darkmyter (using Ultralytics YOLO)
- Ultralytics YOLO 11 + Botsort

**Important**: Run cells in order from top to bottom!

In [1]:
# Cell 1: Setup directories and utilities

from pathlib import Path
import os

BASE_DIR = Path("/content")
REPOS_DIR = BASE_DIR / "repositories"
VIDEOS_DIR = BASE_DIR / "videos"
CLIPS_DIR = BASE_DIR / "clips"
OUTPUT_DIR = BASE_DIR / "output"

for d in [REPOS_DIR, VIDEOS_DIR, CLIPS_DIR, OUTPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def print_status(msg, status="INFO"):
    "Print colored status messages"
    colors = {
        "INFO": "\033[94m",
        "SUCCESS": "\033[92m",
        "WARNING": "\033[93m",
        "ERROR": "\033[91m",
        "RESET": "\033[0m"
    }
    print(f"{colors.get(status, '')}[{status}] {msg}{colors['RESET']}")

print_status("Directory structure created", "SUCCESS")
print(f"Working directory: {BASE_DIR}")

[SUCCESS] Directory structure created
Working directory: /content


In [2]:
# Cell 2: Clone all repositories

import subprocess

REPOSITORIES = {
    "eagle": "https://github.com/nreHieW/Eagle.git",
    "darkmyter": "https://github.com/Darkmyter/Football-Players-Tracking.git",
}

print_status("Cloning repositories...", "INFO")

for name, url in REPOSITORIES.items():
    repo_path = REPOS_DIR / name

    if repo_path.exists():
        print_status(f"{name}: Already exists, skipping", "WARNING")
        continue

    try:
        print_status(f"{name}: Cloning...", "INFO")
        result = subprocess.run(
            ["git", "clone", url, str(repo_path)],
            capture_output=True,
            text=True,
            timeout=300
        )

        if result.returncode == 0:
            print_status(f"{name}: Cloned successfully", "SUCCESS")
        else:
            print_status(f"{name}: Clone failed - {result.stderr[:100]}", "ERROR")

    except Exception as e:
        print_status(f"{name}: Clone failed - {str(e)}", "ERROR")

print_status("Repository cloning complete", "SUCCESS")

[INFO] Cloning repositories...
[INFO] eagle: Cloning...
[SUCCESS] eagle: Cloned successfully
[INFO] darkmyter: Cloning...
[SUCCESS] darkmyter: Cloned successfully
[SUCCESS] Repository cloning complete


In [3]:
# Cell 3: Install dependencies

print_status("Installing dependencies...", "INFO")

!pip install -q torch torchvision torchaudio tracklab
!pip install -q opencv-python numpy scipy pandas scikit-learn matplotlib
!pip install -q ultralytics supervision
!pip install -q gdown Pillow tqdm requests
!pip install -q \
    loguru cython cython_bbox lap onemetric scikit-image tabulate tqdm numpy torch torchvision opencv-python pyyaml yolox
!pip install -q loguru
!pip install onemetric #THIS CELL IS IMPORTANT


print_status("Dependencies installed", "SUCCESS")

[INFO] Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.4/335.4 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [4]:
from ultralytics import YOLO
model = YOLO("yolo11m.pt")
print("Loaded weights from:", getattr(model, "ckpt_path", "unknown path"))

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loaded weights from: yolo11m.pt


In [5]:
# Cell 4: Download videos from Google Drive

!pip install -q gdown

import gdown
from pathlib import Path

# Shared folder ID
FOLDER_ID = "1Cs4kTX6GYwfcpKyDZdqRKBezz49wT7_N"

print_status("Downloading videos from shared folder...", "INFO")

try:
    gdown.download_folder(
        id=FOLDER_ID,
        output=str(VIDEOS_DIR),
        quiet=False,
        use_cookies=False
    )

    # List downloaded videos
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.MP4', '.AVI', '.MOV', '.MKV']
    available_videos = []

    for ext in video_extensions:
        available_videos.extend(list(VIDEOS_DIR.glob(f"*{ext}")))

    if not available_videos:
        print_status("No video files found", "ERROR")
    else:
        print(f"\nDOWNLOADED {len(available_videos)} VIDEO(S)")
        print("="*50)

        for idx, video in enumerate(available_videos, 1):
            size_mb = video.stat().st_size / (1024 * 1024)
            print(f"{idx}. {video.name} ({size_mb:.1f} MB)")

        print("\n")
        print("VIDEO SELECTION")

        # Ask for number of videos
        print("\nHow many videos do you want to evaluate?")
        print(f"  - Enter a number between 1 and {len(available_videos)}")
        print(f"  - Enter 'all' or leave blank to process ALL {len(available_videos)} videos")

        num_selection = input("\nNumber of videos: ").strip().lower()

        VIDEO_PATHS = []

        if not num_selection or num_selection == 'all':
            VIDEO_PATHS = available_videos
            print_status(f"Selected ALL {len(VIDEO_PATHS)} videos", "SUCCESS")
        elif num_selection.isdigit():
            num_videos = int(num_selection)
            if 1 <= num_videos <= len(available_videos):
                if num_videos == len(available_videos):
                    VIDEO_PATHS = available_videos
                else:
                    print(f"\nSelect {num_videos} video(s) from the list above:")
                    print("  - Enter comma-separated numbers (e.g., '1,3,5')")
                    print(f"  - Or enter 'first' to select the first {num_videos} videos")

                    video_selection = input("\nYour selection: ").strip().lower()

                    if video_selection == 'first':
                        VIDEO_PATHS = available_videos[:num_videos]
                    else:
                        try:
                            indices = [int(x.strip()) for x in video_selection.split(',')]
                            if len(indices) != num_videos:
                                print_status(f"Warning: Selected {len(indices)} videos instead of {num_videos}", "WARNING")
                            for idx in indices[:num_videos]:
                                if 1 <= idx <= len(available_videos):
                                    VIDEO_PATHS.append(available_videos[idx - 1])
                        except ValueError:
                            print_status("Invalid input, selecting first videos", "WARNING")
                            VIDEO_PATHS = available_videos[:num_videos]

                print_status(f"Selected {len(VIDEO_PATHS)} video(s)", "SUCCESS")
                for video in VIDEO_PATHS:
                    print(f"  - {video.name}")
            else:
                print_status(f"Invalid number. Must be between 1 and {len(available_videos)}", "ERROR")
        else:
            print_status("Invalid input", "ERROR")

        if not VIDEO_PATHS:
            print_status("No videos selected", "ERROR")

except Exception as e:
    print_status(f"Download failed: {str(e)}", "ERROR")
    print("\nNote: Make sure the folder is set to 'Anyone with the link can view'")

[INFO] Downloading videos from shared folder...


Retrieving folder contents


Processing file 1rdgLjwQwjrrHt0k2v34v9XCGEJYIwjVH FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg02.mp4
Processing file 1fdNwRhMj73wycryjntayMyUx2XNJB1Wr FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg03.mp4
Processing file 107jRObGsfiRFCJXMcXPoZj-kld9hSYtf FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg04.mp4
Processing file 1mlQNwre8ixs6K96UndFU14SPmJHOKH7Q FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg08.mp4
Processing file 1vlOGrvj-X9k6jhN6CHWCM4XAePPivL3I FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg14.mp4
Processing file 1axUbU4cIlVoYgUw5R8YszVP9zN-N3fPl FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg04.mp4
Processing file 1BGpZmffrpdxEA9K6eIQc3ky4mUKJCEsk FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg06.mp4
Processing file 1sGVeVIk9TgOMsGS0ClrzZM8J-hXYruKN FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg07.mp4
Processing file 1_sN4paZaPWCxmgRLIN7Us3AdtFYHtEp- FULL MATCH  Brazil v Mexico  World Cup 2018 720p

Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1rdgLjwQwjrrHt0k2v34v9XCGEJYIwjVH
To: /content/videos/FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg02.mp4
100%|██████████| 85.5M/85.5M [00:01<00:00, 54.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fdNwRhMj73wycryjntayMyUx2XNJB1Wr
To: /content/videos/FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg03.mp4
100%|██████████| 84.8M/84.8M [00:02<00:00, 40.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=107jRObGsfiRFCJXMcXPoZj-kld9hSYtf
To: /content/videos/FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg04.mp4
100%|██████████| 84.6M/84.6M [00:02<00:00, 29.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1mlQNwre8ixs6K96UndFU14SPmJHOKH7Q
To: /content/videos/FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg08.mp4
100%|██████████| 81.3M/81.3M [00:01<00:00, 64.9MB/s]


DOWNLOADED 15 VIDEO(S)
1. FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p-seg06.mp4 (68.4 MB)
2. FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg07.mp4 (91.7 MB)
3. FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg02.mp4 (81.6 MB)
4. FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg04.mp4 (78.4 MB)
5. FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p-seg15.mp4 (62.8 MB)
6. FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg12.mp4 (94.1 MB)
7. FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg03.mp4 (80.9 MB)
8. FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p-seg09.mp4 (70.4 MB)
9. FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg13.mp4 (82.0 MB)
10. FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p-seg03.mp4 (69.1 MB)
11. FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg14.mp4 (77.9 MB)
12. FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg08.mp4 (77.5 MB)
13. FULL MATCH  Croatia 1-1 Czechia  VIP Tactical

In [6]:
# Cell 5: Prepare videos and clips

import cv2
import subprocess

CLIP_DURATION = 60

# Prepare both full videos and clips
FULL_VIDEOS = {}
VIDEO_CLIPS = {}

for VIDEO_PATH in VIDEO_PATHS:
    VIDEO_NAME = VIDEO_PATH.stem

    print(f"\nPREPARING: {VIDEO_NAME}")

    # Store full video path
    FULL_VIDEOS[VIDEO_NAME] = {"full": VIDEO_PATH}

    # Get video info for clip extraction
    cap = cv2.VideoCapture(str(VIDEO_PATH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    cap.release()

    print(f"Duration: {duration:.1f}s | FPS: {fps:.1f} | Frames: {total_frames}")

    # Determine clip positions
    if duration < CLIP_DURATION * 3:
        if duration < CLIP_DURATION:
            CLIPS = [(0, duration, "full")]
            print_status(f"Video shorter than {CLIP_DURATION}s, will use full video", "INFO")
        else:
            CLIPS = [
                (0, CLIP_DURATION, "start"),
                (max(duration - CLIP_DURATION, 0), CLIP_DURATION, "end")
            ]
            print_status("Will extract start and end clips", "INFO")
    else:
        CLIPS = [
            (0, CLIP_DURATION, "start"),
            ((duration - CLIP_DURATION) / 2, CLIP_DURATION, "middle"),
            (duration - CLIP_DURATION, CLIP_DURATION, "end")
        ]
        print_status("Will extract start, middle, and end clips", "INFO")

    # Extract clips
    CLIP_PATHS = {}
    for start_time, clip_dur, position in CLIPS:
        clip_name = f"{VIDEO_NAME}_{position}.mp4"
        clip_path = CLIPS_DIR / clip_name

        cmd = [
            "ffmpeg", "-i", str(VIDEO_PATH),
            "-ss", str(start_time),
            "-t", str(clip_dur),
            "-c", "copy",
            str(clip_path),
            "-y",
            "-loglevel", "error"
        ]

        result = subprocess.run(cmd, capture_output=True)

        if result.returncode == 0 and clip_path.exists():
            CLIP_PATHS[position] = clip_path
            size_mb = clip_path.stat().st_size / (1024 * 1024)
            print_status(f"Clip '{position}' ready ({size_mb:.1f} MB)", "SUCCESS")
        else:
            print_status(f"Failed to extract '{position}' clip", "ERROR")

    VIDEO_CLIPS[VIDEO_NAME] = CLIP_PATHS

print("\n")
print("PREPARATION COMPLETE")
print(f"Prepared {len(VIDEO_PATHS)} video(s) with both full and clip options")


PREPARING: FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p-seg06
Duration: 304.8s | FPS: 25.0 | Frames: 7620
[INFO] Will extract start, middle, and end clips
[SUCCESS] Clip 'start' ready (13.0 MB)
[SUCCESS] Clip 'middle' ready (12.7 MB)
[SUCCESS] Clip 'end' ready (14.3 MB)

PREPARING: FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg07
Duration: 306.0s | FPS: 50.0 | Frames: 15301
[INFO] Will extract start, middle, and end clips
[SUCCESS] Clip 'start' ready (13.0 MB)
[SUCCESS] Clip 'middle' ready (17.8 MB)
[SUCCESS] Clip 'end' ready (20.5 MB)

PREPARING: FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720-seg02
Duration: 305.0s | FPS: 25.0 | Frames: 7626
[INFO] Will extract start, middle, and end clips
[SUCCESS] Clip 'start' ready (14.9 MB)
[SUCCESS] Clip 'middle' ready (15.4 MB)
[SUCCESS] Clip 'end' ready (15.1 MB)

PREPARING: FULL MATCH  Brazil v Mexico  World Cup 2018 720p-seg04
Duration: 306.0s | FPS: 50.0 | Frames: 15301
[INFO] Will extract start, middle, and end c

In [7]:
# Cell: Setup Darkmyter (Original ByteTrack + YOLO - Authentic Implementation)

print_status("Setting up Darkmyter tracking...", "INFO")

import os
import subprocess
from pathlib import Path

darkmyter_dir = REPOS_DIR / "darkmyter"

# Clone original ByteTrack repo (required for authentic Darkmyter)
bytetrack_dir = darkmyter_dir / "ByteTrack"
if not bytetrack_dir.exists():
    print_status("Cloning original ByteTrack repository...", "INFO")
    subprocess.run([
        "git", "clone", "--depth", "1",
        "https://github.com/ifzhang/ByteTrack.git",
        str(bytetrack_dir)
    ], check=True)

# Install ByteTrack dependencies
!pip install -q cython lap cython_bbox

# Download football-specific weights
weights_dir = darkmyter_dir / "yolov8-weights"
weights_dir.mkdir(parents=True, exist_ok=True)

custom_weights = weights_dir / "yolov8l-football-players.pt"
gdrive_id = "12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx"

def download_darkmyter_weights():
    print_status("Downloading Darkmyter football weights...", "INFO")
    try:
        import gdown
        url = f"https://drive.google.com/uc?id={gdrive_id}"
        gdown.download(url, str(custom_weights), quiet=False)
        print_status("Darkmyter weights downloaded", "SUCCESS")
    except Exception as e:
        print_status(f"Failed to download weights: {e}", "ERROR")

if custom_weights.exists():
    try:
        with open(custom_weights, "rb") as f:
            header = f.read(16)
        if header.startswith(b"<"):
            print_status("Weights file is HTML, re-downloading...", "ERROR")
            custom_weights.unlink(missing_ok=True)
            download_darkmyter_weights()
        else:
            print_status("Darkmyter weights already present", "SUCCESS")
    except Exception:
        custom_weights.unlink(missing_ok=True)
        download_darkmyter_weights()
else:
    download_darkmyter_weights()

# Create Darkmyter wrapper
darkmyter_wrapper = darkmyter_dir / "run_darkmyter.py"
darkmyter_wrapper.write_text('''
#!/usr/bin/env python
"""
Darkmyter: YOLOv8 + original ByteTrack (football, notebook-authentic).

This script is a CLI version of the Roboflow "track players with ByteTrack + YOLOv8"
notebook, adapted to output JSON instead of an annotated video.

It:
- Uses yolov8l-football-players.pt if present, else falls back to yolov8x.pt
- Uses original ifzhang/ByteTrack
- Uses football-specific BYTETrackerArgs
- Uses the same format_predictions + match_detections_with_tracks pattern
"""

import argparse
import json
import sys
from dataclasses import dataclass
from pathlib import Path

import cv2
import numpy as np

try:
    from ultralytics import YOLO
    import torch
except ImportError:
    print("Error: ultralytics or torch not installed", file=sys.stderr)
    sys.exit(1)

# Original ByteTrack imports
BYTETRACK_PATH = Path(__file__).resolve().parent / "ByteTrack"
sys.path.insert(0, str(BYTETRACK_PATH))

try:
    from yolox.tracker.byte_tracker import BYTETracker, STrack  # noqa: F401
except ImportError:
    print("Error: ByteTrack repo not found; expected at ./ByteTrack", file=sys.stderr)
    sys.exit(1)

try:
    from onemetric.cv.utils.iou import box_iou_batch
except ImportError:
    print("Error: onemetric not installed (needed for IoU). "
          "Install with: pip install onemetric", file=sys.stderr)
    sys.exit(1)
try:
    from yolox.tracker.byte_tracker import BYTETracker, STrack
except ImportError as e:
    import traceback
    print("Error importing ByteTrack from ./ByteTrack:", e, file=sys.stderr)
    traceback.print_exc()
    sys.exit(1)


#BYTETrackerArgs: football-specific params (from notebook)
@dataclass(frozen=True)
class BYTETrackerArgs:
    track_thresh: float = 0.25
    track_buffer: int = 30
    match_thresh: float = 0.8
    aspect_ratio_thresh: float = 3.0
    min_box_area: float = 1.0
    mot20: bool = False


# Same mapping as in the notebook
IND_TO_CLS = {
    0: "ball",
    1: "goalkeeper",
    2: "player",
    3: "referee",
}


def format_predictions(predictions, with_conf: bool = True) -> np.ndarray:
    """
    Format YOLO detections to ByteTrack format: (x1, y1, x2, y2, conf).

    This mirrors the notebook's function exactly:
        bbox = pred.boxes.xyxy.int().tolist()[0]
        conf = pred.boxes.conf.item()
    """
    frame_detections = []
    for pred in predictions:
        # pred is a ultralytics Results object with a single box
        bbox = pred.boxes.xyxy.int().tolist()[0]  # [x1, y1, x2, y2]
        conf = float(pred.boxes.conf.item())
        if with_conf:
            detection = bbox + [conf]
        else:
            detection = bbox
        frame_detections.append(detection)

    if not frame_detections:
        # shape must be (0, 5) or (0, 4) depending on with_conf
        return np.zeros((0, 5 if with_conf else 4), dtype=float)

    return np.array(frame_detections, dtype=float)


def match_detections_with_tracks(detections, tracks):
    """
    Notebook-authentic matching:

    - Build detections_bboxes using format_predictions(with_conf=False)
    - Build tracks_bboxes from track.tlbr
    - Compute IoU matrix with box_iou_batch
    - For each track, assign its track_id to the best IoU detection if IoU != 0
    """
    if not detections or not tracks:
        return detections

    detections_bboxes = format_predictions(detections, with_conf=False)
    tracks_bboxes = np.array([track.tlbr for track in tracks], dtype=float)

    iou = box_iou_batch(tracks_bboxes, detections_bboxes)  # [num_tracks, num_dets]
    track2detection = np.argmax(iou, axis=1)

    for tracker_index, detection_index in enumerate(track2detection):
        if iou[tracker_index, detection_index] != 0:
            detections[detection_index].tracker_id = tracks[tracker_index].track_id

    return detections


def main():
    parser = argparse.ArgumentParser(
        description="Darkmyter: YOLOv8 + original ByteTrack (football)"
    )
    parser.add_argument("--video", required=True, help="Path to input video")
    parser.add_argument("--output", required=True, help="Path to output JSON file")

    args = parser.parse_args()
    video_path = Path(args.video)
    output_path = Path(args.output)

    if not video_path.exists():
        print(f"Error: video not found: {video_path}", file=sys.stderr)
        sys.exit(1)

    # --- Load YOLO model (football weights if available) -----------------------
    repo_root = Path(__file__).resolve().parent
    custom_weights = repo_root / "yolov8-weights" / "yolov8l-football-players.pt"

    if custom_weights.exists():
        print(f"[Darkmyter] Using football-specific weights: {custom_weights}", file=sys.stderr)
        model = YOLO(str(custom_weights))
        model_name = "yolov8l-football"
        football_specific = True
    else:
        print("[Darkmyter] Football weights not found, using yolov8x.pt", file=sys.stderr)
        model = YOLO("yolov8x.pt")
        model_name = "yolov8x"
        football_specific = False

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"[Darkmyter] Device: {device}", file=sys.stderr)

    # --- Open video -----------------------------------------------------------
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error: cannot open video: {video_path}", file=sys.stderr)
        sys.exit(1)

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps is None or fps <= 0:
        fps = 30.0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.get(cv2.CAP_PROP_FRAME_COUNT) > 0 else -1

    print(f"[Darkmyter] FPS={fps:.1f}, total_frames={total_frames}", file=sys.stderr)

    # --- Initialize ByteTrack (with proper frame_rate) ------------------------
    tracker = BYTETracker(BYTETrackerArgs(), frame_rate=int(round(fps)))

    detections_json = []
    total_tracks = set()
    frame_idx = 0


    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Notebook: detections = yolo_model(frame, verbose=0)[0]
        results = model(frame, verbose=False)[0]

        # "detections" in the notebook is iterable; each element is a single-box Results
        detections_with_tracker = []
        for detection in results:
            detection.tracker_id = ""  # will be filled in after tracking
            detections_with_tracker.append(detection)

        if detections_with_tracker:
            # get trackers with ByteTrack
            bt_input = format_predictions(detections_with_tracker, with_conf=True)

            tracks = tracker.update(
                output_results=bt_input,
                img_info=frame.shape,
                img_size=frame.shape,
            )

            # set tracker_id in yolo detections
            detections_with_tracker = match_detections_with_tracks(
                detections_with_tracker,
                tracks,
            )

            # Convert to JSON rows
            for det in detections_with_tracker:
                if det.tracker_id == "":
                    continue

                # Single box per det
                bbox = det.boxes.xyxy.tolist()[0]
                x1, y1, x2, y2 = map(float, bbox)
                conf = float(det.boxes.conf.item())
                cls_idx = int(det.boxes.cls.item()) if det.boxes.cls is not None else 0

                detections_json.append(
                    {
                        "frame_id": int(frame_idx),
                        "track_id": int(det.tracker_id),
                        "bbox": [x1, y1, x2, y2],
                        "score": conf,
                        "class_id": cls_idx,
                        "class_name": IND_TO_CLS.get(cls_idx, "unknown"),
                    }
                )
                total_tracks.add(int(det.tracker_id))

        if frame_idx % 100 == 0:
            if total_frames > 0:
                pct = 100.0 * frame_idx / total_frames
                print(f"[Darkmyter] Processed {frame_idx}/{total_frames} frames ({pct:.1f}%)",
                      file=sys.stderr)
            else:
                print(f"[Darkmyter] Processed {frame_idx} frames...", file=sys.stderr)

        frame_idx += 1

    cap.release()

    stats = {
        "total_tracks": len(total_tracks),
        "frames_processed": frame_idx,
    }

    full_output = {
        "framework": "Darkmyter",
        "model": model_name,
        "tracker": "ByteTrack (ifzhang/ByteTrack)",
        "tracker_params": {
            "track_thresh": BYTETrackerArgs.track_thresh,
            "track_buffer": BYTETrackerArgs.track_buffer,
            "match_thresh": BYTETrackerArgs.match_thresh,
            "aspect_ratio_thresh": BYTETrackerArgs.aspect_ratio_thresh,
            "min_box_area": BYTETrackerArgs.min_box_area,
        },
        "features": {
            "football_specific": football_specific,
            "original_bytetrack": True,
        },
        "detections": detections_json,
        "statistics": stats,
    }

    output_path.parent.mkdir(parents=True, exist_ok=True)
    with output_path.open("w") as f:
        json.dump(full_output, f, indent=2)

    print(
        f"[Darkmyter] Complete: {len(detections_json)} detections, "
        f"{stats['total_tracks']} tracks. Output saved to: {output_path}",
        file=sys.stderr,
    )


if __name__ == "__main__":
    main()
''')

darkmyter_wrapper.chmod(0o755)
print_status("Darkmyter wrapper created (original ByteTrack)", "SUCCESS")

[INFO] Setting up Darkmyter tracking...
[INFO] Cloning original ByteTrack repository...
  Preparing metadata (setup.py) ... done
[INFO] Downloading Darkmyter football weights...


Downloading...
From (original): https://drive.google.com/uc?id=12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx
From (redirected): https://drive.google.com/uc?id=12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx&confirm=t&uuid=e2a65c2e-964b-41ca-a590-64b4f7f040f3
To: /content/repositories/darkmyter/yolov8-weights/yolov8l-football-players.pt
100%|██████████| 87.6M/87.6M [00:03<00:00, 22.3MB/s]

[SUCCESS] Darkmyter weights downloaded
[SUCCESS] Darkmyter wrapper created (original ByteTrack)


In [8]:
# Patch ByteTrack for NumPy 1.24+ compatibility
import os

bytetrack_path = REPOS_DIR / "darkmyter" / "ByteTrack"

# Files that typically have np.float issues
for root, dirs, files in os.walk(bytetrack_path):
    for file in files:
        if file.endswith('.py'):
            filepath = os.path.join(root, file)
            try:
                with open(filepath, 'r') as f:
                    content = f.read()

                # Replace deprecated numpy aliases
                new_content = content
                new_content = new_content.replace('np.float,', 'float,')
                new_content = new_content.replace('np.float)', 'float)')
                new_content = new_content.replace('np.float]', 'float]')
                new_content = new_content.replace('np.int,', 'int,')
                new_content = new_content.replace('np.int)', 'int)')
                new_content = new_content.replace('np.int]', 'int]')

                if new_content != content:
                    with open(filepath, 'w') as f:
                        f.write(new_content)
                    print(f"Patched: {filepath}")
            except Exception as e:
                pass

print("ByteTrack patched for NumPy compatibility")

Patched: /content/repositories/darkmyter/ByteTrack/yolox/tracker/matching.py
Patched: /content/repositories/darkmyter/ByteTrack/yolox/tracker/byte_tracker.py
Patched: /content/repositories/darkmyter/ByteTrack/yolox/motdt_tracker/matching.py
Patched: /content/repositories/darkmyter/ByteTrack/yolox/motdt_tracker/reid_model.py
Patched: /content/repositories/darkmyter/ByteTrack/yolox/motdt_tracker/motdt_tracker.py
Patched: /content/repositories/darkmyter/ByteTrack/yolox/deepsort_tracker/detection.py
Patched: /content/repositories/darkmyter/ByteTrack/yolox/deepsort_tracker/deepsort.py
Patched: /content/repositories/darkmyter/ByteTrack/tutorials/cstrack/tracker.py
Patched: /content/repositories/darkmyter/ByteTrack/tutorials/cstrack/byte_tracker.py
Patched: /content/repositories/darkmyter/ByteTrack/tutorials/centertrack/byte_tracker.py
Patched: /content/repositories/darkmyter/ByteTrack/tutorials/centertrack/mot_online/matching.py
Patched: /content/repositories/darkmyter/ByteTrack/tutorials/ct

In [18]:
import textwrap
from pathlib import Path
import yaml
import ultralytics

REPOS_DIR = Path("/content/repositories")
ultra_dir = REPOS_DIR / "ultra_trackers"
ultra_dir.mkdir(parents=True, exist_ok=True)


# 1) Choose custom configs for bytetrack + botsort
runner_script = ultra_dir / "run_ultra_yolo_tracker.py"
runner_script.write_text(textwrap.dedent("""\
    #!/usr/bin/env python
    \"\"\"Run Ultralytics YOLO (v5 / v8 / v11 weights) with a chosen tracker and dump JSON tracks.

    Usage:
      python run_ultra_yolo_tracker.py \\
          --video input.mp4 \\
          --output output.json \\
          --weights yolo11m.pt \\
          --tracker botsort
    \"\"\"

    import argparse
    import json
    from pathlib import Path
    from ultralytics import YOLO
    import yaml

    def main():
        parser = argparse.ArgumentParser(description="YOLO + tracker to JSON")
        parser.add_argument("--video", required=True, help="Path to input video")
        parser.add_argument("--output", required=True, help="Path to output JSON")
        parser.add_argument(
            "--weights",
            default="yolo11m.pt",
            help="YOLO weights (e.g., yolov5s.pt, yolov8n.pt, yolo11m.pt, ...)",
        )
        parser.add_argument(
            "--tracker",
            default="botsort",
            choices=["botsort", "bytetrack", "deepsort"],
            help="Which tracker to use",
        )
        parser.add_argument("--conf", type=float, default=0.3,
                    help="Confidence threshold (detector)")
        parser.add_argument("--iou", type=float, default=0.4,
                    help="IOU threshold for NMS (lower = keep more boxes)")
        parser.add_argument("--imgsz", type=int, default=1280,
                    help="Image size for inference")
        parser.add_argument("--max-det", type=int, default=300,
                    help="Maximum detections per image")

        args = parser.parse_args()

        video_path = Path(args.video)
        out_path = Path(args.output)

        if not video_path.exists():
            raise SystemExit(f"Video not found: {video_path}")

        # Load YOLO model
        model = YOLO(args.weights)

        # Get the script's directory for saving custom configs
        ultra_root = Path(__file__).resolve().parent

        # Try to load Ultralytics default configs first
        import ultralytics
        ultra_path = Path(ultralytics.__file__).parent
        tracker_base_path = ultra_path / "cfg" / "trackers"

        # Select the default config path
        if args.tracker == "bytetrack":
            default_cfg_path = tracker_base_path / "bytetrack.yaml"
        elif args.tracker == "botsort":
            default_cfg_path = tracker_base_path / "botsort.yaml"
        else:  # deepsort
            default_cfg_path = tracker_base_path / "deepsort.yaml"

        # Path for our custom config
        custom_cfg_path = ultra_root / f"{args.tracker}_football.yaml"

        # Load and modify the config
        if default_cfg_path.exists():
            # Load the default config
            with open(default_cfg_path, 'r') as f:
                tracker_cfg = yaml.safe_load(f)

            # Modify with football-optimized values from tracklab

            if args.tracker == "botsort":
                tracker_cfg.update({
                    "track_high_thresh": 0.33824964456239337,
                    "new_track_thresh": 0.21144301345190655,
                    "track_buffer": 60,
                    "match_thresh": 0.22734550911325851,
                    "proximity_thresh": 0.5945380911899254,
                    "appearance_thresh": 0.4818211117541298,
                    "cmc_method": "sparseOptFlow",
                    "frame_rate": 30,
                    "lambda_": 0.9896143462366406,
                    "conf_thres": 0.3501265956918775,
                    "with_reid": True
                })

            # Save the modified config to a file
            with open(custom_cfg_path, 'w') as f:
                yaml.dump(tracker_cfg, f)

            # Use the custom config FILE PATH (not the dictionary!)
            tracker_cfg_path = str(custom_cfg_path)
        else:
            # Fallback: just use the default tracker name
            print(f"Warning: Could not find default config at {default_cfg_path}")
            print(f"Using default tracker: {args.tracker}.yaml")
            tracker_cfg_path = f"{args.tracker}.yaml"

        # Run tracking with the config FILE PATH
        results = model.track(
            source=str(video_path),
            tracker=tracker_cfg_path,  # Pass the FILE PATH, not dictionary!
            conf=args.conf,
            iou=args.iou,
            imgsz=args.imgsz,
            max_det=args.max_det,
            stream=True,
            device=0,
            save=False,
            verbose=False,
            persist=True,
            vid_stride=1,
        )

        print(f"Tracking with {args.tracker} on device: {model.device}")

        all_detections = []
        frame_idx = 0

        for r in results:
            boxes = r.boxes
            if boxes is None:
                frame_idx += 1
                continue

            ids = boxes.id
            if ids is None:
                frame_idx += 1
                continue

            xyxy = boxes.xyxy
            confs = boxes.conf
            clses = boxes.cls

            ids = ids.cpu().tolist()
            xyxy = xyxy.cpu().tolist()
            confs = confs.cpu().tolist()
            clses = clses.cpu().tolist()

            for tid, (x1, y1, x2, y2), score, c in zip(ids, xyxy, confs, clses):
                all_detections.append({
                    "frame_id": frame_idx,
                    "track_id": int(tid),
                    "bbox": [float(x1), float(y1), float(x2), float(y2)],
                    "score": float(score),
                    "class_id": int(c),
                })

            frame_idx += 1

        out_path.parent.mkdir(parents=True, exist_ok=True)
        with out_path.open("w") as f:
            json.dump(all_detections, f)

        print(f"Wrote {len(all_detections)} tracked detections to {out_path}")


    if __name__ == "__main__":
        main()
    """))

runner_script.chmod(0o755)
print_status("Created wrapper for Botsort and Bytetrack", "SUCCESS")


[SUCCESS] Created wrapper for Botsort and Bytetrack


In [19]:

# Cell: Setup Eagle with Python 3.13

print_status("Setting up Eagle with Python 3.13...", "INFO")

eagle_dir = REPOS_DIR / "eagle"

# Install Python 3.13 (Eagle's required version)
print_status("Installing Python 3.13...", "INFO")
!apt-get update -qq
!apt-get install -y software-properties-common
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update -qq
!apt-get install -y python3.13 python3.13-venv python3.13-dev python3.13-distutils

# Install pip for Python 3.13
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.13

# Install uv if not already installed
print_status("Installing uv...", "INFO")
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv to PATH
import os
os.environ['PATH'] = f"/root/.local/bin:{os.environ['PATH']}"

# Create Eagle environment with Python 3.13
os.chdir(eagle_dir)
print_status("Creating Eagle environment with Python 3.13...", "INFO")
!uv venv --python python3.13
!uv sync

# Download model weights
print_status("Downloading Eagle model weights...", "INFO")
models_dir = eagle_dir / "eagle" / "models"
if models_dir.exists():
    os.chdir(models_dir)
    !bash get_weights.sh
    os.chdir(eagle_dir)
    print_status("Eagle weights downloaded", "SUCCESS")
else:
    print_status("Eagle models directory not found", "ERROR")

# Create Eagle wrapper that uses Python 3.13

eagle_wrapper = eagle_dir / "run_eagle.py"
eagle_wrapper.write_text('''
#!/usr/bin/env python3
"""
Clean Eagle wrapper that produces a single output file
Consolidates Eagle's multiple outputs into the format expected by the evaluation pipeline
"""

import argparse
import json
import subprocess
import sys
import os
from pathlib import Path
import time


def consolidate_eagle_output(eagle_output_dir, output_format="tracking"):
    """
    Consolidate Eagle's output into a single JSON file

    Args:
        eagle_output_dir: Path to Eagle's output directory
        output_format: "tracking" for standard format, "raw" for Eagle's native format

    Returns:
        Consolidated data dictionary
    """
    # Look for raw_coordinates folder
    coords_dir = eagle_output_dir / "raw_coordinates"
    if not coords_dir.exists():
        coords_dir = eagle_output_dir

    # Find the main coordinates file
    raw_coords_file = coords_dir / "raw_coordinates.json"
    raw_data_file = coords_dir / "raw_data.json"
    processed_file = coords_dir / "processed_data.json"

    # Try different file options in order of preference
    data = None
    source_file = None

    for file_path in [raw_coords_file, processed_file, raw_data_file]:
        if file_path.exists():
            with open(file_path, 'r') as f:
                data = json.load(f)
            source_file = file_path
            print(f"[Eagle] Using {file_path.name} as source", file=sys.stderr)
            break

    if data is None:
        # Try to find any JSON file
        json_files = list(coords_dir.glob("*.json"))
        if json_files:
            with open(json_files[0], 'r') as f:
                data = json.load(f)
            source_file = json_files[0]
            print(f"[Eagle] Using {json_files[0].name} as source", file=sys.stderr)

    if data is None:
        print(f"[Eagle] No output files found in {coords_dir}", file=sys.stderr)
        # Signal error to caller
        raise FileNotFoundError(f"No Eagle JSON outputs found in {coords_dir}")

    # Convert to standard tracking format if requested
    if output_format == "tracking":
        return convert_to_tracking_format(data, source_file)
    else:
        return data


def convert_to_tracking_format(eagle_data, source_file):
    """
    Convert Eagle's format to standard tracking format
    [{"frame_id": N, "track_id": M, "bbox": [x1,y1,x2,y2], "score": S, "class_id": C}, ...]
    """
    tracking_data = []

    # Handle different Eagle output formats
    if isinstance(eagle_data, dict) and all(key.isdigit() for key in eagle_data.keys()):
        # Format: {"0": {...}, "1": {...}, ...} - raw_coordinates.json format
        for frame_str, frame_data in eagle_data.items():
            frame_id = int(frame_str)

            if 'Coordinates' in frame_data:
                coords = frame_data['Coordinates']

                # Process players
                for player_id, player_data in coords.get('Player', {}).items():
                    if 'BBox' in player_data:
                        tracking_data.append({
                            'frame_id': frame_id,
                            'track_id': int(player_id),
                            'bbox': player_data['BBox'],
                            'score': player_data.get('Confidence', 1.0),
                            'class_id': 0  # Player
                        })

                # Process goalkeepers
                for gk_id, gk_data in coords.get('Goalkeeper', {}).items():
                    if 'BBox' in gk_data:
                        tracking_data.append({
                            'frame_id': frame_id,
                            'track_id': int(gk_id),
                            'bbox': gk_data['BBox'],
                            'score': gk_data.get('Confidence', 1.0),
                            'class_id': 1  # Goalkeeper
                        })

    elif isinstance(eagle_data, list):
        # Format: [{...}, {...}, ...] - raw_data.json format
        for frame_id, frame_data in enumerate(eagle_data):
            # Process player entries
            for key in frame_data:
                if key.startswith('Player_') and '_video' in key:
                    player_id = int(key.replace('Player_', '').replace('_video', ''))
                    coords = frame_data.get(key)

                    if coords is not None:
                        # Convert center point to bounding box
                        cx, cy = coords
                        # Estimate bbox (can be adjusted based on typical player size)
                        half_width, half_height = 10, 20

                        tracking_data.append({
                            'frame_id': frame_id,
                            'track_id': player_id,
                            'bbox': [cx - half_width, cy - half_height,
                                   cx + half_width, cy + half_height],
                            'score': 1.0,
                            'class_id': 0  # Player
                        })

                elif key.startswith('Goalkeeper_') and '_video' in key:
                    gk_id = int(key.replace('Goalkeeper_', '').replace('_video', ''))
                    coords = frame_data.get(key)

                    if coords is not None:
                        cx, cy = coords
                        half_width, half_height = 10, 20

                        tracking_data.append({
                            'frame_id': frame_id,
                            'track_id': gk_id,
                            'bbox': [cx - half_width, cy - half_height,
                                   cx + half_width, cy + half_height],
                            'score': 1.0,
                            'class_id': 1  # Goalkeeper
                        })

    return tracking_data


def main():
    parser = argparse.ArgumentParser(description='Eagle wrapper for unified output')
    parser.add_argument('--video', required=True, help='Path to input video')
    parser.add_argument('--output', required=True, help='Path to output JSON file')
    parser.add_argument('--fps', default=20, type=int, help='FPS to process (default: 20)')
    parser.add_argument('--format', choices=['tracking', 'raw'], default='raw',
                       help='Output format: tracking (standard) or raw (Eagle native)')
    args = parser.parse_args()

    video_path = Path(args.video)
    output_path = Path(args.output)

    if not video_path.exists():
        print(f"Error: Video not found: {video_path}", file=sys.stderr)
        sys.exit(1)

    # Ensure output directory exists
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Set up environment
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = "0"

    # Run Eagle
    cmd = [
        "uv", "run", "--python", "python3.13",
        "main.py",
        "--video_path", str(video_path),
        "--fps", str(args.fps),
    ]

    print(f"[Eagle] Processing {video_path.name} at {args.fps} FPS...", file=sys.stderr)
    start = time.time()

    eagle_dir = Path(__file__).parent
    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        cwd=eagle_dir,
        env=env,
    )

    elapsed = time.time() - start
    print(f"[Eagle] Processing took {elapsed:.1f}s", file=sys.stderr)

    if result.returncode != 0:
        print(f"[Eagle] Warning: Process returned {result.returncode}", file=sys.stderr)
        if result.stderr:
            print(f"[Eagle] Stderr: {result.stderr[:500]}", file=sys.stderr)

    # Find Eagle's output directory
    video_stem = video_path.stem
    eagle_output_base = eagle_dir / "output"
    eagle_output_dir = eagle_output_base / video_stem

    if not eagle_output_dir.exists():
        # Try to find directory with video name in it
        for d in eagle_output_base.iterdir():
            if d.is_dir() and video_stem in d.name:
                eagle_output_dir = d
                break

    if not eagle_output_dir.exists():
        print(f"[Eagle] Error: Could not find output directory for {video_stem}", file=sys.stderr)
        # Write empty output
        with open(output_path, 'w') as f:
            json.dump([], f)
        sys.exit(1)

    # Convert Eagle output into single file
    try:
        print(f"[Eagle] Consolidating output from {eagle_output_dir}", file=sys.stderr)
        consolidated_data = consolidate_eagle_output(eagle_output_dir, args.format)
    except Exception as e:
        print(f"[Eagle] Error while consolidating Eagle output: {e}", file=sys.stderr)
        with open(output_path, "w") as f:
            json.dump([], f)
        sys.exit(1)

    # Write converted output
    with open(output_path, 'w') as f:
        json.dump(consolidated_data, f, indent=2)

    # Report statistics
    if isinstance(consolidated_data, list):
        if consolidated_data and 'frame_id' in consolidated_data[0]:
            # Tracking format
            frame_ids = set(d['frame_id'] for d in consolidated_data)
            track_ids = set(d['track_id'] for d in consolidated_data)
            print(f"[Eagle] Output: {len(consolidated_data)} detections", file=sys.stderr)
            print(f"[Eagle] Frames: {len(frame_ids)} ({min(frame_ids)}-{max(frame_ids)})", file=sys.stderr)
            print(f"[Eagle] Tracks: {len(track_ids)} unique IDs", file=sys.stderr)
        else:
            print(f"[Eagle] Output: {len(consolidated_data)} frames", file=sys.stderr)
    elif isinstance(consolidated_data, dict):
        print(f"[Eagle] Output: {len(consolidated_data)} frames (raw format)", file=sys.stderr)

    print(f"[Eagle] Saved to: {output_path}", file=sys.stderr)
    sys.exit(0)


if __name__ == "__main__":
    main()

''')

eagle_wrapper.chmod(0o755)
print_status("Eagle FULL capability wrapper created", "SUCCESS")

[INFO] Setting up Eagle with Python 3.13...
[INFO] Installing Python 3.13...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
software-properties-common is already the newest version (0.99.22.9).
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.
Repository: 'deb https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu/ jammy main'
Description:
This PPA contains more recent Python versions packaged for Ubuntu.

Disclaimer: there's no guarantee of timely updates in case of security problems or other issues. If you want to use them in a security-or-otherwise-critical environment (say, on a production server), you do so at your own risk.

Update Note
Please use this repository instead of ppa:fkrull/deadsnakes.

Reporting Issues

Issues can be re

In [12]:
from google.colab import drive
from pathlib import Path

drive.mount("/content/drive")

# Base folder on Drive where all results will go
BASE_DIR = Path("/content/drive/MyDrive/tracklab_eval")  # change name if you like
OUTPUT_DIR = BASE_DIR / "results"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("Results will be saved under:", OUTPUT_DIR)


Mounted at /content/drive
Results will be saved under: /content/drive/MyDrive/tracklab_eval/results


In [ ]:
# Cell: Final System Evaluation

import time
import json
import subprocess
import os
from pathlib import Path
"""
SYSTEM_CONFIGS = {
        "eagle": {
        "path": REPOS_DIR / "eagle",
        "script": "run_eagle.py",
        "python": "python3.13",
    },
    "yolo11_botsort": {
        "path": REPOS_DIR / "ultra_trackers",
        "script": "run_ultra_yolo_tracker.py",
        "args": ["--weights", "yolo11m.pt", "--tracker", "botsort",],
    },
    "darkmyter": {
        "path": REPOS_DIR / "darkmyter",
        "script": "run_darkmyter.py",
    },
}
"""

# System configurations
SYSTEM_CONFIGS = {
    "eagle": {
        "path": REPOS_DIR / "eagle",
        "script": "run_eagle.py",
        "python": "python3.13",
    },
    "yolo11_botsort": {
        "path": REPOS_DIR / "ultra_trackers",
        "script": "run_ultra_yolo_tracker.py",
        "args": ["--weights", "yolo11m.pt", "--tracker", "botsort",],
    },
    "darkmyter": {
        "path": REPOS_DIR / "darkmyter",
        "script": "run_darkmyter.py",
    },
}

# Ask user for processing mode
print("\n" + "="*50)
print("EVALUATION MODE SELECTION")
print("="*50)
print("\nHow do you want to evaluate the videos?")
print("  1. Use clips (faster - 60s segments)")
print("  2. Use full videos (comprehensive but slower)")

mode_choice = input("\nEnter your choice (1 or 2): ").strip()
USE_CLIPS = mode_choice != '2'

if USE_CLIPS:
    print_status("Mode: CLIP-BASED EVALUATION", "INFO")
    ALL_VIDEOS = VIDEO_CLIPS
    eval_type = "clips"
else:
    print_status("Mode: FULL VIDEO EVALUATION", "INFO")
    ALL_VIDEOS = FULL_VIDEOS
    eval_type = "full"

position_to_number = {"start": 1, "middle": 2, "end": 3, "full": 1}

def run_system_on_video(system_name, system_config, video_name, segment_name, video_path):
    """Run a tracking system on a video or clip"""

    if USE_CLIPS:
        segment_number = position_to_number.get(segment_name, 1)
        output_dir = OUTPUT_DIR / video_name / "clips" / str(segment_number) / system_name
    else:
        output_dir = OUTPUT_DIR / video_name / "full" / system_name

    output_dir.mkdir(parents=True, exist_ok=True)

    if USE_CLIPS:
        print_status(f"Running {system_name} on {video_name}/clip_{segment_number}...", "INFO")
    else:
        print_status(f"Running {system_name} on {video_name} (full video)...", "INFO")

    start_time = time.time()
    output_file = output_dir / f"{system_name}_output.json"
    system_path = system_config.get("path", REPOS_DIR)

    # Build command
    if system_name == "eagle":
        cmd = [
            "uv", "run", "--python", system_config.get("python", "python3.13"),
            "run_eagle.py",
            "--video", str(video_path),
            "--output", str(output_file),
        ]
    else:
        cmd = [
            "python", system_config["script"],
            "--video", str(video_path),
            "--output", str(output_file),
        ] + [str(extra) for extra in system_config.get("args", [])]

    try:
        #Not using a timeout
        timeout = 10000
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            cwd=str(system_path),
        )

        elapsed = time.time() - start_time

        if result.returncode == 0 and output_file.exists():
            try:
                with open(output_file) as f:
                    data = json.load(f)

                if isinstance(data, list):
                    num_detections = len(data)
                elif isinstance(data, dict):
                    num_detections = sum(
                        len(dets) if isinstance(dets, list) else 0
                        for dets in data.values()
                    )
                else:
                    num_detections = 0

                print_status(
                    f"{system_name}: SUCCESS - {num_detections} detections in {elapsed:.1f}s",
                    "SUCCESS",
                )
                return {
                    "success": True,
                    "time": elapsed,
                    "output": str(output_file),
                    "detections": num_detections,
                }

            except json.JSONDecodeError as e:
                print_status(f"{system_name}: Invalid JSON", "ERROR")
                return {"success": False, "time": elapsed, "error": f"Invalid JSON: {e}"}
        else:
            error_msg = result.stderr[-500:] if result.stderr else "Unknown error"
            print_status(f"{system_name}: FAILED", "ERROR")
            print(f"Error: {error_msg}")
            return {"success": False, "time": elapsed, "error": error_msg}

    except subprocess.TimeoutExpired:
        print_status(f"{system_name}: TIMEOUT", "ERROR")
        return {"success": False, "time": timeout, "error": "Timeout"}

    except Exception as e:
        print_status(f"{system_name}: EXCEPTION - {str(e)}", "ERROR")
        return {"success": False, "time": time.time() - start_time, "error": str(e)}

def save_progress(all_results, eval_type):
    "Save current progress to disk (in Drive, via OUTPUT_DIR)"
    progress_file = OUTPUT_DIR / f"progress_{eval_type}.json"
    with open(progress_file, "w") as f:
        json.dump(all_results, f, indent=2)
    return progress_file

def load_progress(eval_type):
    "Load existing progress if available"
    progress_file = OUTPUT_DIR / f"progress_{eval_type}.json"
    if progress_file.exists():
        try:
            with open(progress_file) as f:
                return json.load(f)
        except Exception:
            return {}
    return {}

print(f"STARTING {eval_type.upper()} EVALUATION\n")

all_results = load_progress(eval_type)
if all_results:
    print_status(f"Loaded existing progress with {len(all_results)} videos", "INFO")
    print("Videos already processed:")
    for video_name in all_results.keys():
        print(f"  - {video_name}")

    # Ask if user wants to continue or restart
    print("\nDo you want to:")
    print("  1. Continue from where you left off")
    print("  2. Start fresh (delete existing progress)")
    continue_choice = input("\nEnter your choice (1 or 2): ").strip()

    if continue_choice == '2':
        all_results = {}
        print_status("Starting fresh evaluation", "INFO")

for video_name, segments in ALL_VIDEOS.items():
    print(f"\nVIDEO: {video_name}")

    video_results = all_results.get(video_name, {})

    for segment_name, video_path in segments.items():
        if USE_CLIPS:
            segment_number = position_to_number.get(segment_name, 1)
            segment_key = f"clip_{segment_number}"
            print(f"\nProcessing clip {segment_number} ({segment_name})...")
        else:
            segment_key = "full"
            print(f"\nProcessing full video...")

        segment_results = video_results.get(segment_key, {})
        video_results[segment_key] = segment_results

        for system_name, system_config in SYSTEM_CONFIGS.items():
            # Skip if already processed successfully
            if system_name in segment_results and segment_results[system_name].get("success"):
                print_status(f"{system_name}: Already completed successfully", "SKIP")
                continue
            elif system_name in segment_results:
                print_status(f"{system_name}: Retrying previous failure", "RETRY")

            result = run_system_on_video(
                system_name, system_config, video_name, segment_name, video_path
            )
            segment_results[system_name] = result

            # Save progress after each system completes (to Drive)
            all_results[video_name] = video_results
            progress_file = save_progress(all_results, eval_type)
            print_status(f"Progress saved to {progress_file.name}", "SAVE")

        successful = sum(1 for r in segment_results.values() if r.get("success", False))
        total = len(segment_results)

        if USE_CLIPS:
            print(f"\nClip summary: {successful}/{total} systems succeeded")
        else:
            print(f"\nVideo summary: {successful}/{total} systems succeeded")

    # Save per-video summary
    summary_file = OUTPUT_DIR / video_name / f"summary_{eval_type}.json"
    summary_file.parent.mkdir(parents=True, exist_ok=True)
    with open(summary_file, "w") as f:
        json.dump(video_results, f, indent=2)
    print_status(f"Video summary saved to {summary_file.name}", "SAVE")

# Save overall summary
overall_summary = OUTPUT_DIR / f"overall_summary_{eval_type}.json"
with open(overall_summary, "w") as f:
    json.dump(all_results, f, indent=2)

print("\n" + "=" * 60)
print(f"{eval_type.upper()} EVALUATION COMPLETE")
print("=" * 60)

system_stats = {
    sys: {"success": 0, "total": 0, "avg_time": [], "avg_detections": []}
    for sys in SYSTEM_CONFIGS.keys()
}

for video_results in all_results.values():
    for segment_results in video_results.values():
        for system_name, result in segment_results.items():
            if system_name in system_stats:
                system_stats[system_name]["total"] += 1
                if result.get("success", False):
                    system_stats[system_name]["success"] += 1
                    if "time" in result:
                        system_stats[system_name]["avg_time"].append(result["time"])
                    if "detections" in result:
                        system_stats[system_name]["avg_detections"].append(result["detections"])

print("\nSystem Success Rates:")
print("-" * 40)
for system_name, stats in system_stats.items():
    if stats["total"] > 0:
        success_rate = (stats["success"] / stats["total"]) * 100
        print(f"\n{system_name}:")
        print(f"  Success Rate: {stats['success']}/{stats['total']} ({success_rate:.1f}%)")
        if stats["avg_time"]:
            avg_time = sum(stats["avg_time"]) / len(stats["avg_time"])
            print(f"  Avg Time: {avg_time:.1f}s")
        if stats["avg_detections"]:
            avg_det = sum(stats["avg_detections"]) / len(stats["avg_detections"])
            print(f"  Avg Detections: {avg_det:.0f}")

print(f"\nResults Directory: {OUTPUT_DIR}")
print(f"Overall Summary: {overall_summary}")
print(f"Progress File: {OUTPUT_DIR / f'progress_{eval_type}.json'}")

total_expected = len(ALL_VIDEOS) * len(next(iter(ALL_VIDEOS.values())))
total_processed = sum(len(video_results) for video_results in all_results.values())

if total_processed == total_expected:
    print("\n")
    print("ALL VIDEOS PROCESSED SUCCESSFULLY!")
    print("Progress file kept for reference.")
else:
    print("\n")
    print(f"PARTIAL COMPLETION: {total_processed}/{total_expected} segments processed")
    print("Run the script again to continue from where you left off.")


EVALUATION MODE SELECTION

How do you want to evaluate the videos?
  1. Use clips (faster - 60s segments)
  2. Use full videos (comprehensive but slower)

Enter your choice (1 or 2): 2
[INFO] Mode: FULL VIDEO EVALUATION
STARTING FULL EVALUATION


VIDEO: FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p-seg06

Processing full video...
[INFO] Running eagle on FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p-seg06 (full video)...


In [ ]:
cd /content/path/to/Eagle  # eagle_dir
uv run main.py --video_path /full/path/to/one/clip.mp4 --fps 20


In [ ]:
"""
Evaluation Cell for Football Player Tracking Systems

"""
import json
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print_status("Starting comprehensive tracking evaluation...", "INFO")

# DATA STRUCTURES

@dataclass
class Detection:
    """Universal detection representation"""
    frame_id: int
    track_id: int
    bbox: List[float]  # [x1, y1, x2, y2]
    confidence: float = 1.0
    class_id: int = 0

    # Eagle-specific
    transformed_coords: Optional[Tuple[float, float]] = None
    is_goalkeeper: bool = False

    # Metadata
    source_system: str = ""
    raw_data: Dict = field(default_factory=dict)


@dataclass
class SystemData:
    """Container for system tracking data"""
    name: str
    frames: Dict[int, List[Detection]]
    metadata: Dict[str, Any]
    source_file: str
    clip_name: str = ""



# DATA LOADERS


class UniversalLoader:
    """Load tracking data from different systems"""
    @staticmethod
    def load_eagle(filepath: Path) -> SystemData:
        """Load Eagle data (handles both raw_coordinates and standard format)"""
        with open(filepath, "r") as f:
            data = json.load(f)

        from collections import defaultdict
        frames = defaultdict(list)
        metadata = {"system": "Eagle", "features": []}


        # CASE 1: Eagle raw_coordinates format
        if isinstance(data, dict) and data:
            all_keys = list(data.keys())
            if all(k.isdigit() for k in all_keys):
                metadata["features"] = ["field_coordinates", "goalkeeper_detection", "keypoints"]
                metadata["format"] = "raw_coordinates"

                # iterate all frames, not just one
                for frame_str in sorted(all_keys, key=lambda s: int(s)):
                    frame_data = data[frame_str]
                    frame_id = int(frame_str)

                    coords = frame_data.get("Coordinates", {})

                    # Players
                    for player_id, player_data in coords.get("Player", {}).items():
                        if "BBox" not in player_data:
                            continue

                        tc = player_data.get("Transformed_Coordinates")
                        if isinstance(tc, (list, tuple)) and len(tc) >= 2:
                            transformed_coords = (float(tc[0]), float(tc[1]))
                        else:
                            transformed_coords = None

                        det = Detection(
                            frame_id=frame_id,
                            track_id=int(player_id),
                            bbox=player_data["BBox"],
                            confidence=player_data.get("Confidence", 1.0),
                            class_id=0,
                            transformed_coords=transformed_coords,
                            is_goalkeeper=False,
                            source_system="Eagle",
                            raw_data=player_data,
                        )
                        frames[frame_id].append(det)

                    # Goalkeepers
                    for gk_id, gk_data in coords.get("Goalkeeper", {}).items():
                        if "BBox" not in gk_data:
                            continue

                        tc = gk_data.get("Transformed_Coordinates")
                        if isinstance(tc, (list, tuple)) and len(tc) >= 2:
                            transformed_coords = (float(tc[0]), float(tc[1]))
                        else:
                            transformed_coords = None

                        det = Detection(
                            frame_id=frame_id,
                            track_id=int(gk_id),
                            bbox=gk_data["BBox"],
                            confidence=gk_data.get("Confidence", 1.0),
                            class_id=1,
                            transformed_coords=transformed_coords,
                            is_goalkeeper=True,
                            source_system="Eagle",
                            raw_data=gk_data,
                        )
                        frames[frame_id].append(det)

                    # Keypoints metadata
                    if "Keypoints" in frame_data:
                        metadata.setdefault("keypoints", []).append(
                            {"frame": frame_id, "points": frame_data["Keypoints"]}
                        )


        # CASE 2: already in standard tracking format--
        elif isinstance(data, list):
            metadata["format"] = "standard"
            for det_dict in data:
                det = Detection(
                    frame_id=det_dict["frame_id"],
                    track_id=det_dict["track_id"],
                    bbox=det_dict["bbox"],
                    confidence=det_dict.get("score", 1.0),
                    class_id=det_dict.get("class_id", 0),
                    source_system="Eagle",
                )
                frames[det.frame_id].append(det)

        return SystemData(
            name="Eagle",
            frames=dict(frames),
            metadata=metadata,
            source_file=str(filepath),
        )
    @staticmethod
    def load_darkmyter(filepath: Path) -> SystemData:
        """Load Darkmyter data (YOLOv8 with ByteTrack)"""
        with open(filepath, 'r') as f:
            data = json.load(f)

        frames = defaultdict(list)
        metadata = {"system": "Darkmyter"}

        # Check if it's the full format with metadata
        if isinstance(data, dict) and 'framework' in data:
            metadata.update({
                "model": data.get('model', 'yolov8'),
                "tracker": data.get('tracker', 'ByteTrack'),
                "features": data.get('features', {}),
                "statistics": data.get('statistics', {})
            })
            detections = data.get('detections', [])
        else:
            # Simple list format
            detections = data if isinstance(data, list) else []

        for det_dict in detections:
            det = Detection(
                frame_id=det_dict['frame_id'],
                track_id=det_dict['track_id'],
                bbox=det_dict['bbox'],
                confidence=det_dict['score'],
                class_id=det_dict['class_id'],
                source_system="Darkmyter"
            )
            frames[det.frame_id].append(det)

        return SystemData(
            name="Darkmyter",
            frames=dict(frames),
            metadata=metadata,
            source_file=str(filepath)
        )

    @staticmethod
    def load_yolo11(filepath: Path, tracker_name: str = "YOLO11") -> SystemData:
        """Load YOLO11 data (with BotSort or ByteTrack)"""
        with open(filepath, 'r') as f:
            data = json.load(f)

        frames = defaultdict(list)
        metadata = {
            "system": tracker_name,
            "model": "YOLOv11",
            "tracker": "BotSort" if "botsort" in tracker_name.lower() else "ByteTrack"
        }

        # Handle both list and dict formats
        detections = data if isinstance(data, list) else data.get('detections', [])

        for det_dict in detections:
            det = Detection(
                frame_id=det_dict['frame_id'],
                track_id=det_dict['track_id'],
                bbox=det_dict['bbox'],
                confidence=det_dict['score'],
                class_id=det_dict['class_id'],
                source_system=tracker_name
            )
            frames[det.frame_id].append(det)

        return SystemData(
            name=tracker_name,
            frames=dict(frames),
            metadata=metadata,
            source_file=str(filepath)
        )



# EVALUATION METRICS


class TrackingEvaluator:
    """Evaluate tracking performance"""

    def __init__(self, iou_threshold: float = 0.5):
        self.iou_threshold = iou_threshold

    def calculate_iou(self, bbox1: List[float], bbox2: List[float]) -> float:
        """Calculate Intersection over Union"""
        x1_min, y1_min, x1_max, y1_max = bbox1
        x2_min, y2_min, x2_max, y2_max = bbox2

        x_inter_min = max(x1_min, x2_min)
        y_inter_min = max(y1_min, y2_min)
        x_inter_max = min(x1_max, x2_max)
        y_inter_max = min(y1_max, y2_max)

        if x_inter_max < x_inter_min or y_inter_max < y_inter_min:
            return 0.0

        inter_area = (x_inter_max - x_inter_min) * (y_inter_max - y_inter_min)
        area1 = (x1_max - x1_min) * (y1_max - y1_min)
        area2 = (x2_max - x2_min) * (y2_max - y2_min)
        union_area = area1 + area2 - inter_area

        return inter_area / union_area if union_area > 0 else 0.0

    def evaluate_system(self, system_data: SystemData) -> Dict:
        """Evaluate a single system"""
        total_detections = sum(len(dets) for dets in system_data.frames.values())
        unique_tracks = set()
        confidence_values = []

        for detections in system_data.frames.values():
            for det in detections:
                unique_tracks.add(det.track_id)
                confidence_values.append(det.confidence)

        # Track continuity analysis
        track_lifetimes = defaultdict(list)
        for frame_id, detections in system_data.frames.items():
            for det in detections:
                track_lifetimes[det.track_id].append(frame_id)

        # Calculate fragmentations
        fragmentations = 0
        for track_id, frame_list in track_lifetimes.items():
            frame_list = sorted(frame_list)
            for i in range(1, len(frame_list)):
                if frame_list[i] - frame_list[i-1] > 1:
                    fragmentations += 1

        # Eagle-specific metrics
        eagle_metrics = {}
        if system_data.name == "Eagle" and system_data.metadata.get('format') == 'raw_coordinates':
            goalkeeper_count = sum(1 for dets in system_data.frames.values()
                                 for det in dets if det.is_goalkeeper)
            with_coords = sum(1 for dets in system_data.frames.values()
                             for det in dets if det.transformed_coords)
            eagle_metrics = {
                'goalkeeper_detections': goalkeeper_count,
                'detections_with_field_coords': with_coords,
                'has_keypoints': 'keypoints' in system_data.metadata
            }

        return {
            'system': system_data.name,
            'clip': system_data.clip_name,
            'total_frames': len(system_data.frames),
            'total_detections': total_detections,
            'unique_tracks': len(unique_tracks),
            'avg_detections_per_frame': total_detections / len(system_data.frames) if system_data.frames else 0,
            'avg_confidence': np.mean(confidence_values) if confidence_values else 0,
            'std_confidence': np.std(confidence_values) if confidence_values else 0,
            'fragmentations': fragmentations,
            'avg_track_lifetime': np.mean([len(frames) for frames in track_lifetimes.values()]) if track_lifetimes else 0,
            **eagle_metrics
        }

    def compare_systems(self, sys1: SystemData, sys2: SystemData) -> Dict:
        """Compare two systems on overlapping frames"""
        common_frames = set(sys1.frames.keys()) & set(sys2.frames.keys())

        if not common_frames:
            return {
                'comparison': f"{sys1.name} vs {sys2.name}",
                'clip': sys1.clip_name,
                'common_frames': 0,
                'message': 'No overlapping frames'
            }

        matches = 0
        total_iou = 0
        sys1_only = 0
        sys2_only = 0

        for frame_id in common_frames:
            dets1 = sys1.frames[frame_id]
            dets2 = sys2.frames[frame_id]

            matched2 = set()

            for d1 in dets1:
                best_iou = 0
                best_match = None

                for i, d2 in enumerate(dets2):
                    if i in matched2:
                        continue
                    iou = self.calculate_iou(d1.bbox, d2.bbox)
                    if iou > best_iou:
                        best_iou = iou
                        best_match = i

                if best_iou >= self.iou_threshold:
                    matches += 1
                    total_iou += best_iou
                    matched2.add(best_match)
                else:
                    sys1_only += 1

            sys2_only += len(dets2) - len(matched2)

        return {
            'comparison': f"{sys1.name} vs {sys2.name}",
            'clip': sys1.clip_name,
            'common_frames': len(common_frames),
            'matched_detections': matches,
            'avg_iou': total_iou / matches if matches > 0 else 0,
            f'{sys1.name}_only': sys1_only,
            f'{sys2.name}_only': sys2_only,
            'match_rate': matches / (matches + sys1_only + sys2_only) if (matches + sys1_only + sys2_only) > 0 else 0
        }



# LOAD DATA FROM YOUR EXACT STRUCTURE


def load_tracking_outputs_from_structure(base_dir: Path):
    """
    Load tracking outputs from your directory structure:

    /content/output/
        <MATCH_NAME>/
            clips/
                1/
                    darkmyter/darkmyter_output.json
                    eagle/eagle_output.json
                    yolo11_botsort/yolo11_botsort_output.json
                    yolo11_bytetrack/yolo11_bytetrack_output.json
    """
    systems = []
    loader = UniversalLoader()

    if not base_dir.exists():
        print_status(f"Base directory does not exist: {base_dir}", "ERROR")
        return systems

    print_status(f"Scanning directory: {base_dir}", "INFO")

    # Treat *every* subdirectory of base_dir as a match directory
    for match_dir in sorted(base_dir.iterdir()):
        if not match_dir.is_dir():
            continue

        match_name = match_dir.name
        print_status(f"Found match: {match_name}", "INFO")

        clips_dir = match_dir / "clips"
        if not clips_dir.exists():
            print_status(f"  No clips directory in {match_name}", "WARNING")
            continue

        # Process each clip folder (e.g., "1", "2", ...)
        for clip_dir in sorted(clips_dir.iterdir()):
            if not clip_dir.is_dir():
                continue

            clip_name = clip_dir.name
            print_status(f"  Processing clip {clip_name}...", "INFO")

            # Eagle
            eagle_dir = clip_dir / "eagle"
            if eagle_dir.exists():
                eagle_file = eagle_dir / "eagle_output.json"
                if eagle_file.exists():
                    try:
                        system_data = loader.load_eagle(eagle_file)
                        system_data.clip_name = f"{match_name}_clip_{clip_name}"
                        systems.append(system_data)
                        print_status(
                            f"    ✓ Eagle: {len(system_data.frames)} frames",
                            "SUCCESS"
                        )
                    except Exception as e:
                        print_status(f"    ✗ Eagle failed: {e}", "ERROR")

            #  Darkmyter
            darkmyter_dir = clip_dir / "darkmyter"
            if darkmyter_dir.exists():
                darkmyter_file = darkmyter_dir / "darkmyter_output.json"
                if darkmyter_file.exists():
                    try:
                        system_data = loader.load_darkmyter(darkmyter_file)
                        system_data.clip_name = f"{match_name}_clip_{clip_name}"
                        systems.append(system_data)
                        print_status(
                            f"    ✓ Darkmyter: {len(system_data.frames)} frames",
                            "SUCCESS"
                        )
                    except Exception as e:
                        print_status(f"    ✗ Darkmyter failed: {e}", "ERROR")

            #  YOLO11 BotSort
            yolo11_botsort_dir = clip_dir / "yolo11_botsort"
            if yolo11_botsort_dir.exists():
                yolo11_botsort_file = yolo11_botsort_dir / "yolo11_botsort_output.json"
                if yolo11_botsort_file.exists():
                    try:
                        system_data = loader.load_yolo11(
                            yolo11_botsort_file,
                            "YOLO11-BotSort"
                        )
                        system_data.clip_name = f"{match_name}_clip_{clip_name}"
                        systems.append(system_data)
                        print_status(
                            f"    ✓ YOLO11-BotSort: {len(system_data.frames)} frames",
                            "SUCCESS"
                        )
                    except Exception as e:
                        print_status(f"    ✗ YOLO11-BotSort failed: {e}", "ERROR")

            #  YOLO11 ByteTrack
            yolo11_bytetrack_dir = clip_dir / "yolo11_bytetrack"
            if yolo11_bytetrack_dir.exists():
                yolo11_bytetrack_file = (
                    yolo11_bytetrack_dir / "yolo11_bytetrack_output.json"
                )
                if yolo11_bytetrack_file.exists():
                    try:
                        system_data = loader.load_yolo11(
                            yolo11_bytetrack_file,
                            "YOLO11-ByteTrack"
                        )
                        system_data.clip_name = f"{match_name}_clip_{clip_name}"
                        systems.append(system_data)
                        print_status(
                            f"    ✓ YOLO11-ByteTrack: {len(system_data.frames)} frames",
                            "SUCCESS"
                        )
                    except Exception as e:
                        print_status(f"    ✗ YOLO11-ByteTrack failed: {e}", "ERROR")

    return systems


    # Summary Statistics Table
    ax6 = plt.subplot(2, 4, 6)
    ax6.axis('tight')
    ax6.axis('off')

    if not eval_df.empty:
        summary_data = []
        for _, row in system_summary.iterrows():
            summary_data.append([
                row['system'],
                f"{int(row['total_frames'])}",
                f"{int(row['total_detections'])}",
                f"{row['unique_tracks']:.0f}",
                f"{row['avg_confidence']:.3f}"
            ])

        table = ax6.table(cellText=summary_data,
                         colLabels=['System', 'Frames', 'Detections', 'Avg Tracks', 'Avg Conf'],
                         cellLoc='center',
                         loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1, 1.5)

    ax6.set_title('Aggregated Summary Statistics', pad=20)

    plt.suptitle('Football Tracking Systems Evaluation', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    return fig



# MAIN EVALUATION EXECUTION


# Set your base output directory
OUTPUT_DIR = Path("/content/output")  # Adjust this to your path

# Load tracking data from your structure
print_status("Loading tracking outputs...", "INFO")
systems = load_tracking_outputs_from_structure(OUTPUT_DIR)

if not systems:
    print_status("No tracking data found! Please check your output directory.", "ERROR")
    print_status(f"Expected structure: {OUTPUT_DIR}/FULL_MATCH_*/clips/*/system_name/", "INFO")
else:
    print_status(f"Successfully loaded {len(systems)} system outputs", "SUCCESS")

    # Evaluate each system
    evaluator = TrackingEvaluator(iou_threshold=0.5)
    evaluation_results = []

    print("\n" + "="*60)
    print("INDIVIDUAL SYSTEM EVALUATION")
    print("="*60)

    for system in systems:
        metrics = evaluator.evaluate_system(system)
        evaluation_results.append(metrics)

        print(f"\n{system.name} - {system.clip_name}:")
        print(f"  Frames: {metrics['total_frames']}")
        print(f"  Detections: {metrics['total_detections']}")
        print(f"  Unique tracks: {metrics['unique_tracks']}")
        print(f"  Avg confidence: {metrics['avg_confidence']:.3f}")

    # Pairwise comparisons within each clip
    comparison_results = []

    # Group systems by clip
    clips = defaultdict(list)
    for system in systems:
        clips[system.clip_name].append(system)

    print("\n" + "="*60)
    print("PAIRWISE SYSTEM COMPARISON")
    print("="*60)

    for clip_name, clip_systems in clips.items():
        if len(clip_systems) > 1:
            print(f"\nClip: {clip_name}")

            for i in range(len(clip_systems)):
                for j in range(i+1, len(clip_systems)):
                    comparison = evaluator.compare_systems(clip_systems[i], clip_systems[j])
                    comparison_results.append(comparison)

                    if comparison['common_frames'] > 0:
                        print(f"  {comparison['comparison']}:")
                        print(f"    Common frames: {comparison['common_frames']}")
                        print(f"    Match rate: {comparison['match_rate']:.3f}")

    # Create visualizations
    print("\n" + "="*60)
    print("GENERATING EVALUATION PLOTS")
    print("="*60)

    fig = create_evaluation_plots(evaluation_results, comparison_results)

    # Save results
    eval_df = pd.DataFrame(evaluation_results)
    eval_df.to_csv('tracking_evaluation_results.csv', index=False)
    print_status("Results saved to tracking_evaluation_results.csv", "SUCCESS")

    if comparison_results:
        comp_df = pd.DataFrame(comparison_results)
        comp_df.to_csv('tracking_comparison_results.csv', index=False)
        print_status("Comparisons saved to tracking_comparison_results.csv", "SUCCESS")

    # Display summary
    print("\n")
    print("EVALUATION SUMMARY")

    # Aggregate by system
    system_avg = eval_df.groupby('system').agg({
        'avg_confidence': 'mean',
        'unique_tracks': 'mean',
        'fragmentations': 'sum'
    }).reset_index()

    if not system_avg.empty:
        best_confidence = system_avg.loc[system_avg['avg_confidence'].idxmax()]
        best_tracks = system_avg.loc[system_avg['unique_tracks'].idxmax()]
        best_continuity = system_avg.loc[system_avg['fragmentations'].idxmin()]

        print(f"\nBest confidence: {best_confidence['system']} ({best_confidence['avg_confidence']:.3f})")
        print(f"Most tracks: {best_tracks['system']} ({best_tracks['unique_tracks']:.0f} avg tracks)")
        print(f"Best continuity: {best_continuity['system']} ({best_continuity['fragmentations']:.0f} total fragmentations)")

print_status("Evaluation complete!", "SUCCESS")

In [ ]:
# ===================== DEBUG EAGLE OUTPUT FOR ONE CLIP ===================== #

from pathlib import Path
import json
from collections import Counter


# --- Configure which match + clip to inspect ---
MATCH_NAME = "FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720"
CLIP_ID = "2"   # e.g. "1", "2", "3", ...

BASE_OUTPUT_DIR = Path("/content/output")
eagle_json = BASE_OUTPUT_DIR / MATCH_NAME / "clips" / CLIP_ID / "eagle" / "eagle_output.json"

print("=== PATH CHECK ===")
print("Eagle JSON path:", eagle_json)
print("Exists:", eagle_json.exists())
if not eagle_json.exists():
    raise FileNotFoundError(eagle_json)

# --- Inspect raw JSON structure ------------------------------------------- #
with eagle_json.open("r") as f:
    data = json.load(f)

print("\n=== RAW JSON STRUCTURE ===")
print("Top-level type:", type(data))

if isinstance(data, dict):
    # Expect raw_coordinates.json style: {"0": {...}, "1": {...}, ...}
    keys = list(data.keys())
    print("Number of top-level keys (frames):", len(keys))
    print("First 10 keys:", keys[:10])

    frame_ids = []
    det_per_frame = []
    for frame_str, frame_data in data.items():
        frame_id = int(frame_str)
        frame_ids.append(frame_id)

        coords = frame_data.get("Coordinates", {})
        n_players = len(coords.get("Player", {}))
        n_gks = len(coords.get("Goalkeeper", {}))
        det_per_frame.append(n_players + n_gks)

    print("Frame id range:", min(frame_ids), "to", max(frame_ids))
    print("Detections per frame: min =", min(det_per_frame),
          "max =", max(det_per_frame),
          "mean =", sum(det_per_frame) / len(det_per_frame))

elif isinstance(data, list):
    print("List length:", len(data))
    if data and isinstance(data[0], dict) and "frame_id" in data[0]:
        # Our tracking format: [{"frame_id": ..., "track_id": ..., ...}, ...]
        frame_ids = [d["frame_id"] for d in data]
        counts = Counter(frame_ids)
        print("Number of distinct frame_ids:", len(counts))
        print("First 10 (frame_id: count):", list(counts.items())[:10])

print("\n=== LOADER VIEW (UniversalLoader.load_eagle) ===")
try:
    loader = UniversalLoader()
except NameError:
    raise RuntimeError("UniversalLoader is not defined. Run the evaluation cell first.")

system_data = loader.load_eagle(eagle_json)
system_data.clip_name = f"{MATCH_NAME}_clip_{CLIP_ID}"

print("System name:", system_data.name)
print("Number of frame entries in system_data.frames:", len(system_data.frames))

if system_data.frames:
    frame_ids_loaded = sorted(system_data.frames.keys())
    print("First 10 frame ids in loader:", frame_ids_loaded[:10])

    total_dets = sum(len(v) for v in system_data.frames.values())
    print("Total detections (loader):", total_dets)

    unique_tracks = sorted({d.track_id for lst in system_data.frames.values() for d in lst})
    print("Number of unique tracks:", len(unique_tracks))
    print("Sample of track_ids:", unique_tracks[:20])

print("\n=== EVALUATOR METRICS FOR THIS CLIP (EAGLE ONLY) ===")
try:
    evaluator = TrackingEvaluator()
    metrics = evaluator.evaluate_system(system_data)
    for k, v in metrics.items():
        print(f"{k}: {v}")
except NameError:
    print("TrackingEvaluator not defined. Run the evaluation cell first.")


In [ ]:
# Cell 9: Download results

from google.colab import files
import shutil

print_status("Creating archive...", "INFO")

archive_name = "tracking_results"
archive_path = BASE_DIR / archive_name

shutil.make_archive(str(archive_path), 'zip', OUTPUT_DIR)

print_status("Downloading...", "SUCCESS")
files.download(f"{archive_path}.zip")

print_status("Complete!", "SUCCESS")